In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# Widgets as Output: A Demo

In this notebook we learn how to react to change-events on input-fields.

Sometimes you need to react to user input directly, not after the user pressed the `call`-button. For example the user has a `file_selector`-widget, and selects some input-data. So maybe for this selected case some other available input-widgets are not used anymore, so you would like to disable them, to guide the user to the important parts, or you would like to set some defaults for other fields dedicated  to the selected input file. To do this, you can add `on_change`-handler to your model already in the declarative part, so in the `init`-method of our model. You can add an `on_change`-event handler to each widget. If a widget doesn't have a declared `on_change`-attribute, no action will be executed when you change the value of the widget. To declare an `on_change`-action you have 3 different options:
 * single string, containing the python code to be executed in case the value has changed 
 * array of strings, which will be concatenated with "\n", and the remaining String will be executed in case the value has changed
 * directly connect to a method of our model, without additional parameters
 
For the first two variants  you can write your code directly in the declaration of your widget. Your string containing python-code will be executed each time the value has been changed. In this execution environment the new value is available as the local variable `new_value`, while the old value is available as the local variable `old_value`. All other widgets of the same tab are directly addressable by their `name`-attribute. In addition the whole model is available over the local variable  `model`. So this event-handling is tightly coupled to the widget, which raised this `on_change`-event. This makes it ideal for shorter locally bound logic. Example:

``` python
    build_qubo_parameter=[
        {'name':"P", 'type':'int_bounded', 'value':0, 'min':0, 'max':10, 
         'label':'$P$', 'description':'Please select an integer value between 0 and 10',
         'on_change':[
             'text = area.value',
             'text += "value of P changed from %d to %d\\n" % (old_value, new_value)',        
             'area.value = text'
                     ]},                                    
        {'name':"area", 'type':'text_area', 'value':'', 
         'label':'area', 'description':'Please add some long text here'}
    ]
```

Each time the integer value of the widget `P` changes, the new value will be added to the value of the `text_area`-widget below. The `on_change`-handler is implemented as a list of strings.

The local bounding makes this ideal for smaller code fragments, but is not so good for longer python code, which you might want to program with syntax highlighting, and maybe also need to call as an `on_change`-handler for different widgets. In this case you can directly link the `on_change`-event to a method of your model. A major disadvantage is only, that the linked method is always called without parameters. For example:

``` python
    {'name':"i_c", 'type':'int_bounded', 'value':0, 'min':0, 'max':10, 
     'label':'$i_{c}$', 'description':'Please select an integer value between 0 and 10',
     'on_change':self.do_something_each_time_c_changes} 
```

The method `do_something_each_time_c_changes` from the model is now called each time the value of widget $i_c$ changes.

To better understand the procedure, it is advisable to run through the notebook below.
Also try the different `on_change`-events in the dashboard which will be created upon executing the following cell.

In [ ]:
from dadk.Optimizer import *

class Demo_Widgets_Events_Model(OptimizerModel):
    
    ##################################################################################################################
    def __init__(self, persistent_file="Demo_Widgets.dao"):
    ##################################################################################################################
        
        OptimizerModel.__init__(
            self,
            name = 'Demo_Widgets',
            load_parameter=[                
                {'name':'s', 'type':'select', 'value':' ', 'options':[' ', 'a', 'b', 'c'], 
                 'label':'$s$', 'description':'Please select a single value from the list',
                 'on_change':'''
if new_value == "a":         
    i_a.value = 10            
    i_b.value =  0           
    i_c.value =  0                           
elif new_value == "b":       
    i_a.value =  0           
    i_b.value = 10            
    i_c.value =  0                                
elif new_value == "c":        
    i_a.value =  0            
    i_b.value =  0           
    i_c.value = 10           
                             '''},
                {'name':"i_a", 'type':'int_bounded', 'value':0, 'min':0, 'max':10, 
                 'label':'$i_{a}$', 'description':'Please select an integer value between 0 and 10',
                 'on_change':[
                     'print("Hello")',
                     'print("world!")']},                                    
                {'name':"i_b", 'type':'int_bounded', 'value':0, 'min':0, 'max':10, 
                 'label':'$i_{b}$', 'description':'Please select an integer value between 0 and 10',
                 'on_change':'print(new_value)'},                                    
                {'name':"i_c", 'type':'int_bounded', 'value':0, 'min':0, 'max':10, 
                 'label':'$i_{c}$', 'description':'Please select an integer value between 0 and 10',
                 'on_change':self.do_something_each_time_c_changes}                                 
            ],
            
            
            build_qubo_toolbar_parameter=[
                {'name':"x", 'type':'int_bounded', 'value':0, 'min':0, 'max':100, 
                 'label':'x', 'description':'I_don\'t know :-)',
                 'on_change':'print(new_value)'},
                {'name':"button_p", 'type':'button',  
                 'label':'Click me! (to disable/enable "P")', 
                 'on_click':'''
print("'button_p' clicked")
model.Optimizer.gui_build().P.disabled = not model.Optimizer.gui_build().P.disabled
                 '''},
                {'name':"button_area", 'type':'button',  
                 'label':'Click me! (to hide/show "area")', 
                 'on_click':'''
print("'button_area' clicked")
model.Optimizer.gui_build().area.visible = not model.Optimizer.gui_build().area.visible
                 '''},
            ],
            
            build_qubo_parameter=[
                {'name':"P", 'type':'int_bounded', 'value':0, 'min':0, 'max':10, 
                 'label':'$P$', 'description':'Please select an integer value between 0 and 10',
                 'on_change':[
                     'text = area.value',
                     'text += f"Value of P changed from {old_value} to {new_value}\\n"',        
                     'area.value = text'
                             ]},                                    
                {'name':"area", 'type':'text_area', 'value':'', 
                 'label':'area', 'description':'Please add some long text here'}
            ],
            
            solve_on_focus='print("focus now on tab \\"solve\\"")',
            
            solve_toolbar_parameter=[
                {'name':"y", 'type':'button',  
                 'label':'Click me!', 
                 'on_click':'print("button on tab \\"solve\\" clicked")'},
            ],
            
            
            persistent_file = persistent_file)
        
    def do_something_each_time_c_changes(self):
        print("someting changed")
        
            
    ##################################################################################################################
    def load(self, s=None, i_a=None, i_b=None, i_c=None, silent=False):
    ##################################################################################################################
        
        print("s = \"%s\"" % s)  
        print("i_a = %d" % i_a)
        print("i_b = %d" % i_b)
        print("i_c = %d" % i_c)
        
        self.LOGGER.debug("message DEBUG")
        self.LOGGER.info("message INFO")
        self.LOGGER.error("message ERROR")
        self.LOGGER.fatal("message FATAL")
        
        self.Optimizer.set_calculated_build_qubo_parameter(P=max(i_a, i_b, i_c))
                    

    ##################################################################################################################
    def build_qubo(self, P=None, area=None, silent=False):
    ##################################################################################################################

        print("P = %d" % P)  
        print("area = \"%s\"" % area)
    
        self.HQ = BinPol().add_term(1, (0,1)).add_term(1, (2,3))
        
        if not silent:
            print(f'''QUBO created
    Number of bits:    {self.HQ.N: 5d}
    More infos to be added ...''')
            

    ##################################################################################################################
    def prep_result(self, solution_list:SolutionList, silent=False):
    ##################################################################################################################

        configuration = solution_list.min_solution.configuration

        self.energy = self.HQ.compute(configuration)
        
        if not silent:
            print(f'''Solution energies:
    HQ:        {self.energy: 12.2f}
    More infos to be added ...''')
            
            
            
# log_format="'%(asctime)s - %(name)s - %(levelname)s - %(message)s'"
# log_format="%(levelname)s - %(message)s"
# log_level=logging.DEBUG logging.INFO

optimizer = Optimizer(Demo_Widgets_Events_Model(), auto_load=False, log_level=logging.ERROR, log_format="%(levelname)s - %(message)s")